<h2>1.Data processing</h2>

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing as pre
from sklearn.model_selection import train_test_split

# ML Libraries
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import random

columns = ['id','nb_words_title','nb_words_content','pp_uniq_words','pp_stop_words','pp_uniq_non-stop_words',
            'nb_links','nb_outside_links','nb_images','nb_videos','ave_word_length','nb_keywords','category',
            'nb_mina_mink','nb_mina_maxk','nb_mina_avek','nb_maxa_mink','nb_maxa_maxk','nb_maxa_avek','nb_avea_mink',
            'nb_avea_maxk','nb_avea_avek','nb_min_linked','nb_max_linked','nb_ave_linked','weekday','dist_topic_0',
            'dist_topic_1','dist_topic_2','dist_topic_3','dist_topic_4','subj','polar','pp_pos_words','pp_neg_words',
            'pp_pos_words_in_nonneutral','ave_polar_pos','min_polar_pos','max_polar_pos','ave_polar_neg','min_polar_neg',
            'max_polar_neg','subj_title','polar_title'
          ]

train_data = pd.read_csv('train.csv', delimiter =' ', names = columns)
train_data_target = pd.read_csv('train-targets.csv')
train_data_target = train_data_target.set_index('Id')
validation_set = pre.scale(pd.read_csv('test-val.csv',delimiter = ' ',names=columns).set_index('id'))



#test_set = pd.read_csv('test-val.csv', delimiter = ' ')



util_train_data = pre.scale(train_data.drop('id', axis=1))
util_train_data = pd.DataFrame(util_train_data, columns = columns[1:])

train_data, test_set, target_train, target_test = train_test_split(
                                                        util_train_data,
                                                        train_data_target,
                                                        random_state = 0)



In [2]:
new_target_train =np.log(target_train['Prediction'])
new_target_test = np.log(target_test['Prediction'])

<h2>2.Evaluation Metric</h2>

In [3]:
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y
     
    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

<h2>3. Data mining </h2>

In [4]:
from sklearn.ensemble import GradientBoostingRegressor

opti_rate = 0.075151515

clf2 = GradientBoostingRegressor(learning_rate = opti_rate, random_state = 1)
clf2.fit(train_data, new_target_train)
prediction_GB = clf2.predict(test_set)
#rmsle(new_target_test,prediction_GB) # 0.03554735

In [72]:

import keras

from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(20, input_shape= (43,)))
model.add(Dropout(0.4))
model.add(Activation('relu'))
model.add(Dense(20))
model.add(Activation('relu',name= 'good_layer'))
model.add(Dropout(0.4))
model.add(Dense(1))

model.compile(loss='mse', optimizer='rmsprop')



model.fit(np.array(train_data),new_target_train, epochs = 30, batch_size=1000)
#prediction_NN = model.predict(np.array(test_set))
#rmsle(np.array(new_target_test),prediction_NN)

Epoch 1/30
3750/3750 [==============================] - 0s 4us/step - loss: 53.0885
Epoch 2/30
3750/3750 [==============================] - 0s 7us/step - loss: 48.5212
Epoch 3/30
3750/3750 [==============================] - 0s 10us/step - loss: 45.3360
Epoch 4/30
3750/3750 [==============================] - 0s 16us/step - loss: 42.8986
Epoch 5/30
3750/3750 [==============================] - 0s 8us/step - loss: 40.3025
Epoch 6/30
3750/3750 [==============================] - 0s 9us/step - loss: 38.0471
Epoch 7/30
3750/3750 [==============================] - 0s 8us/step - loss: 36.1867
Epoch 8/30
3750/3750 [==============================] - 0s 12us/step - loss: 34.1533
Epoch 9/30
3750/3750 [==============================] - 0s 9us/step - loss: 31.6673
Epoch 10/30
3750/3750 [==============================] - 0s 13us/step - loss: 29.5875
Epoch 11/30
3750/3750 [==============================] - 0s 8us/step - loss: 27.7468
Epoch 12/30
3750/3750 [==============================] - 0s 7us/step -

In [85]:
model_add = Model(inputs = model.input,
                  outputs = model.get_layer("good_layer").output)

model_add.compile(loss = 'mse', optimizer = 'rmsprop')

output_neural = model_add.predict(np.array(train_data))

In [84]:
output_neural

(3750L, 20L)

In [86]:
import lightgbm
clf1 = lightgbm.LGBMRegressor()

clf1.fit(output_neural, np.array(target_train))

C:\Users\vdherbemont\AppData\Local\Continuum\Anaconda2\envs\Python27\lib\site-packages\lightgbm\basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
C:\Users\vdherbemont\AppData\Local\Continuum\Anaconda2\envs\Python27\lib\site-packages\lightgbm\basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
       max_bin=255, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=10,
       n_jobs=-1, num_leaves=31, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [91]:
#Pipe pour test

def pipe_kaggle(data_set, target = None):
    neural_results = model_add.predict(np.array(data_set))
    prediction = clf1.predict(neural_results)
    if target is None:
        return(prediction)
    else:
        return(np.sqrt(np.square(np.log(prediction + 1) - np.log(target + 1)).mean()))

pipe_kaggle(test_set,target = np.array(target_test))

1.0511300719284979

In [96]:
results = pipe_kaggle(validation_set)

(pd.DataFrame((np.round(results)).astype('int'))).to_csv('results.csv')



In [53]:
clf1 = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

NameError: name 'xgboost' is not defined